In [1]:
cd /home/novella/PycharmProjects/stock-price-trend-prediction-remote-magi

/home/novella/PycharmProjects/stock-price-trend-prediction-remote-magi


In [10]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from enum import Enum
from sklearn.cluster import MeanShift, KMeans
import numpy as np
import pickle

import src.utils.lob_util as lbu
import src.config as co
import src.constants as cst

np.random.seed(0)
plt.rcParams["figure.figsize"] = (16,9)

print(os.getcwd())

/home/novella/PycharmProjects/stock-price-trend-prediction-remote-magi


In [11]:
PATH_NAME = "data/LOBSTER_6/unzipped/"
STOCKS_FNAME = ["_data_dwn_48_332__{}_2021-07-01_2021-08-06_10", "_data_dwn_48_332__{}_2020-03-01_2020-04-03_10"]
# STOCKS_PAIRS = [("AAPL", "TSLA"), ("ZM", "AAWW"), ("AGNC", "LYFT")]
STOCKS       = ["SOFI", "NFLX", "CSCO", "WING", "SHLS", "LSTR"]

class Period(Enum):
    JULY = 0
    MARCH = 1

class Granularity(Enum):
    SEC = 1
    MIN = 60
    TEN_MIN = int(60*10)
    HALF_HOUR = int(60*30)
    HOUR = 60*60
    DAY = int(60*60*6.5)
    TWO_DAYS = int(60*60*6.5*2)
    THREE_DAYS = int(60*60*6.5*3)
    FIVE_DAYS = int(60*60*6.5*5)

In [13]:
MID_PRICES = {Period.JULY:None}

for scen in Period:
    daily_midprice = pd.DataFrame()
    RETURNS_FNAME = "data/LOBSTER_6/price_return_df_5_scen{}.dat".format(scen.value)

    if False: # file exists?

        for i, i_stock in enumerate(STOCKS):
            print("Processing stock", i, i_stock)

            stock_path = PATH_NAME + STOCKS_FNAME[scen.value].format(i_stock)
                
            tra_days = os.listdir(stock_path)
            tra_days.sort()

            out_df = lbu.from_folder_to_unique_df(
                stock_path,
                level=10,
                granularity=cst.Granularity.Sec1,
                boundaries_purge=0
            )
            out_df = out_df.fillna(method="ffill")
            daily_midprice[i_stock] = (out_df.loc[:, "psell1"] + out_df.loc[:, "pbuy1"]) / (2 * 10000)
            
            print("HAS CORRECT SHAPE?", out_df.shape[0], out_df.shape[0] == 60*60*6.5*25)
            out_df = None
        
        MID_PRICES[scen] = daily_midprice
        
        with open(RETURNS_FNAME, "wb") as f:
             pickle.dump(daily_midprice, f)
    else:
        print("Loading pickle...")
        with open(RETURNS_FNAME, "rb") as f:
            daily_midprice = pickle.load(f)
        
        MID_PRICES[scen] = daily_midprice

Loading pickle...
Loading pickle...


In [5]:
# daily_midprice = mids[0]  # 0 is JULY
# print(daily_midprice.shape[0])
# print(60*60*6.5*25)
# print("60*60*6.5*25, where 6.5 is the hours in a trading day, 25 days")
# daily_midprice

In [31]:

df = MID_PRICES[Period.JULY]

df.drop(df.index[(np.where((df.index > '2021-08-03') & (df.index < '2021-08-05')))[0]], inplace=True)
MID_PRICES[Period.JULY] = df

def get_trading_day(df, day, gran=Granularity.HOUR):
    return df[gran.value*day:gran.value*(day+1)]


i = 0
while True:
    a = get_trading_day(MID_PRICES[Period.JULY], day=i, gran=Granularity.DAY)
    print(a.index[0], a.index[-1])
    i += 1

2021-07-01 09:30:00 2021-07-01 15:59:59
2021-07-02 09:30:00 2021-07-02 15:59:59
2021-07-06 09:30:00 2021-07-06 15:59:59
2021-07-07 09:30:00 2021-07-07 15:59:59
2021-07-08 09:30:01 2021-07-09 09:30:00
2021-07-09 09:30:01 2021-07-12 09:30:00
2021-07-12 09:30:01 2021-07-13 09:30:00
2021-07-13 09:30:01 2021-07-14 09:30:00
2021-07-14 09:30:01 2021-07-15 09:30:00
2021-07-15 09:30:01 2021-07-16 09:30:00
2021-07-16 09:30:01 2021-07-19 09:30:00
2021-07-19 09:30:01 2021-07-20 09:30:00
2021-07-20 09:30:01 2021-07-21 09:30:00
2021-07-21 09:30:01 2021-07-22 09:30:00
2021-07-22 09:30:01 2021-07-23 09:30:00
2021-07-23 09:30:01 2021-07-26 09:30:00
2021-07-26 09:30:01 2021-07-27 09:30:00
2021-07-27 09:30:01 2021-07-28 09:30:00
2021-07-28 09:30:01 2021-07-29 09:30:00
2021-07-29 09:30:01 2021-07-30 09:30:00
2021-07-30 09:30:01 2021-08-02 09:30:00
2021-08-02 09:30:01 2021-08-05 09:30:00
2021-08-05 09:30:01 2021-08-06 09:30:00
2021-08-06 09:30:01 2021-08-06 15:59:59


IndexError: index 0 is out of bounds for axis 0 with size 0

In [197]:
def eval_norm_mid(daily_midprice, gran=1, smooth=1):
    midprice_ret = daily_midprice.iloc[::gran.value, :]
    midprice_ret = midprice_ret / midprice_ret.iloc[0, :]
    midprice_ret = midprice_ret.rolling(smooth).mean()
    return midprice_ret

def eval_returns(daily_midprice, gran=Granularity.HOUR):
    norm = eval_norm_mid(daily_midprice, gran=Granularity.HOUR.SEC).iloc[::gran.value, :]
    rets = (norm - norm.shift(-1)) / norm.shift(-1)
    rets = rets.dropna()
    return rets

In [198]:
def increment_volatiltuy(df_jul, df_mar, gran=Granularity.HOUR):
    period_stats = pd.DataFrame()
    ret_jul = eval_returns(df_jul, gran)
    ret_mar = eval_returns(df_mar, gran)

    period_stats["vol_jul"] = ret_jul.describe().loc['std']
    period_stats["vol_mar"] = ret_mar.describe().loc['std']

    period_stats["perc_vol_%"] = ((period_stats["vol_mar"] - period_stats["vol_jul"]) / period_stats["vol_jul"])*100  # how much more volatile was MARCH wrt to JULY
    stats = period_stats.describe()

    me = round(stats.loc['mean', 'perc_vol_%'])
    st = round(stats.loc['std', 'perc_vol_%'])
    print("covid period is {}+/-{}% more volatile ({}) than the july 2021 period.".format(me, st, gran))

increment_volatiltuy(MID_PRICES[Period.JULY], MID_PRICES[Period.MARCH], gran=Granularity.HOUR)
increment_volatiltuy(MID_PRICES[Period.JULY], MID_PRICES[Period.MARCH], gran=Granularity.MIN)
increment_volatiltuy(MID_PRICES[Period.JULY], MID_PRICES[Period.MARCH], gran=Granularity.DAY)
increment_volatiltuy(MID_PRICES[Period.JULY], MID_PRICES[Period.MARCH], gran=Granularity.TWO_DAYS)
increment_volatiltuy(MID_PRICES[Period.JULY], MID_PRICES[Period.MARCH], gran=Granularity.THREE_DAYS)
increment_volatiltuy(MID_PRICES[Period.JULY], MID_PRICES[Period.MARCH], gran=Granularity.FIVE_DAYS)

covid period is 316+/-209% more volatile (Granularity.HOUR) than the july 2021 period.
covid period is 832+/-521% more volatile (Granularity.MIN) than the july 2021 period.
covid period is 297+/-139% more volatile (Granularity.DAY) than the july 2021 period.
covid period is 301+/-192% more volatile (Granularity.TWO_DAYS) than the july 2021 period.
covid period is 346+/-239% more volatile (Granularity.THREE_DAYS) than the july 2021 period.
covid period is 816+/-710% more volatile (Granularity.FIVE_DAYS) than the july 2021 period.


In [199]:
def plot_mids(mids_df):
    mids_df.index = [str(ind) for ind in mids_df.index]
    x_axis = list(mids_df.index)
    fig, ax = plt.subplots(figsize=(13, 8))

    ax = mids_df.plot(ax = ax)

    # SCEWNARIO = ('2021-07-22', '2021-07-29', '2021-08-06')
    # SCEWNARIO = ('2020-03-20', '2020-03-27', '2020-04-03')
    # yli = ax.get_ylim()
    # ax.vlines(x=x_axis.index(SCEWNARIO[0])+13, ymin=yli[0], ymax=yli[1], color='black') # 13 mezz'ore in 6.5 ore
    # ax.vlines(x=x_axis.index(SCEWNARIO[1])+13, ymin=yli[0], ymax=yli[1], color='black')
    # ax.vlines(x=x_axis.index(SCEWNARIO[2])+13, ymin=yli[0], ymax=yli[1], color='black')
    # plt.xticks(rotation=10)

    plt.ylabel("midprice (normalized)")
    plt.xlabel("date")
    # plt.ylim([.4, 1.5])
    ax.legend(loc='upper left')
    plt.tight_layout()
    # fig.savefig('scenario-mar.pdf', dpi=400)

In [200]:
plot_mids(eval_norm_mid(MID_PRICES[Period.MARCH], gran=Granularity.TEN_MIN))

<IPython.core.display.Javascript object>

In [201]:
def dataset_distr(window_size_backward, window_size_forward, stock, period, scaler, is_plot=False):
    exp = pd.DataFrame()
    exp["mid"] = MID_PRICES[period].loc[:,[stock]]
    exp["mid+t"] = exp["mid"] #.rolling(window_size_forward ).mean().shift(-window_size_forward+1)
    exp["mid-t"] = exp["mid"].rolling(window_size_backward).mean()

    exp["mid+t-inst"] = exp["mid"].shift(-window_size_forward+1)
    exp["mid-t-inst"] = exp["mid"]

    exp = exp[(window_size_backward - 1):-(window_size_forward - 1)]

    exp["change"] = (exp["mid+t"] - exp["mid-t"]) / exp["mid-t"]
    exp["ret"] = (exp["mid+t-inst"] - exp["mid-t-inst"]) / exp["mid-t-inst"]

    ratio_mu, ratio_si = exp["change"].mean(), exp["change"].std()
    sig_0 = np.abs(exp["change"]).mean()
    print(sig_0)
    th_u = 0.0002 # 0+ratio_si*scaler
    th_d = -0.0002 # 0-ratio_si*scaler

    # >>>> PLOTTING <<<<<
    if is_plot:
        exp.index = [str(ind) for ind in exp.index]
        exp["change"].plot(color="blue")
        #exp["ret"].plot(color="blue")

        plt.axhline(y=0, color="red")
        plt.axhline(y=th_u,     color="green")
        plt.axhline(y=th_d,     color="green")

        plt.axhline(y=sig_0,     color="orange")
        plt.axhline(y=-sig_0,    color="orange")

        plt.show()

    exp["pred"] = np.ones (exp.shape[0])
    exp["pred"] = np.where(exp["change"] > th_u, 2, exp["pred"])
    exp["pred"] = np.where(exp["change"] < th_d, 0, exp["pred"])

    # labels 0 (down), 1 (stable), 2 (down)
    return counter_labels(exp["pred"])

In [34]:
from collections import Counter

class Label(Enum):
    UP = 2.
    STABLE = 1.
    DOWN = 0.

def counter_labels(data):
    counter = Counter(data)
    tot = sum(counter.values())
    pers = []
    for k in Label:
        val = counter[k.value]
        per = round(100*val/tot, 2)
        pers.append(per)
        # print("> label {} has {} ({}%) classes".format(k.name, val, per))
    return pers

def similarity_score(vals):
    a, b, c = vals[0], vals[1], vals[2]
    abs_diffs = [abs(a - b), abs(a - c), abs(b - c)]
    return sum(abs_diffs) / 3

In [203]:
%matplotlib notebook
wb, wf = int(10), int(10)
dataset_distr(wb, wf, "AAPL", Period.JULY, .5, is_plot=False)

8.32105568163085e-05


[4.3, 91.25, 4.45]

In [32]:
MEMO = {}

def bal_dat(x, window_size_backward=100, window_size_forward = 100):
    change = 'change-{}-{}'.format(window_size_backward, window_size_forward)

    if change not in MEMO:
        sss = []
        for s in STOCKS:
            exp = pd.DataFrame()
            exp["mid"] = MID_PRICES[Period.JULY].loc[:,[s]]

            ba = exp["mid"].rolling(window_size_backward).mean().iloc[window_size_backward-1:] # first window_size_backward-1 are NAN due to rolling mean
            fo = exp["mid"].iloc[window_size_backward:].rolling(window_size_forward).mean().iloc[window_size_forward-1:]    # first window_size_backward-1 are NAN due to rolling mean
            ba = ba.iloc[:-window_size_forward]
            fo, ba = fo.values, ba.values

            # exp = exp.iloc[window_size_backward:-window_size_forward+1]  # check + 1
            assert(fo.shape == ba.shape)

            sss.append((ba-fo)/fo)
        MEMO[change] = np.hstack(sss)

    th_u, th_d = x, -x

    PREDS = np.ones (MEMO[change].shape[0])
    PREDS = np.where(MEMO[change] > th_u, 2, PREDS)
    PREDS = np.where(MEMO[change] < th_d, 0, PREDS)

    # labels 0 (down), 1 (stable), 2 (down)
    return counter_labels(PREDS)

In [39]:
# TODAY QUI
bal_dat(0.00024, window_size_backward=100, window_size_forward = 100)

[32.96, 33.16, 33.88]

In [191]:
import numpy as np
from scipy.optimize import differential_evolution

def optimize_balancing(window_size_backward=100, window_size_forward = 100):
    def target_function(x):
        val = bal_dat(x[0], window_size_backward, window_size_forward)
        return np.sum(np.square(np.array(val) - np.array([33, 33, 33])))  # minimize the squared distance

    result = differential_evolution(target_function, [(0,.01)])

    if result.success:
        print(f"Optimal x: {result.x[0]}, f(x): {bal_dat(result.x[0], window_size_backward, window_size_forward)}")
    else:
        print("Optimization failed.")
    return result.x[0], bal_dat(result.x[0], window_size_backward, window_size_forward)

# wins = [10, 50, 100]
# for s in STOCKS:
#     out = optimize_balancing(100, 10, s)
#     print("DONE", s)

# K = 100, 10, 50
# AAPL 0.00067, 0.000207, 0.00046
# TSLA 0.0011, 0.000317, 0.00075
# ZM 0.0011, 0.00023, 0.000743
# AAWW 0.0013, 0.000112211, 0.000747
# AGCN 0.001, 0.000110, 0.0005692
# LYFT 0.001, 0.00024, 0.000829

# per stock sono pressoché costanti

# print(np.mean([0.00067, 0.0011, 0.0011, 0.0013, 0.001, 0.001]))
# for s in STOCKS:
#     print(s, bal_dat(0.001, 10, 10, s))
# bal_dat(0.000207, 10, 10, "AAPL")  # 0.000317


# 6 stock [AAPL, ZM...]
# 6 exp, per modello

# W- = 100, W+ [10, 50, 100]
# W+ = 10 , W- [10, 50, 100]

# concat dei simboli e poi ricerca dell'alfa che bilancia
# un alfa per ognuno dei 6 casi? or take the AVG.
# bal_dat(0.0002, 100, 100)

for wp in [10, 50, 100]:
    for wm in [10, 50, 100]:
        print(wm, wp)
        out = optimize_balancing(wm, wp)

10 10
Optimal x: 0.0002051275234977723, f(x): [33.66, 33.35, 32.98]
50 10
Optimal x: 0.00046934783354042175, f(x): [33.82, 33.38, 32.8]
100 10
Optimal x: 0.0006895166096171209, f(x): [33.95, 33.35, 32.7]
10 50
Optimal x: 0.0004859623000321228, f(x): [33.76, 33.32, 32.91]
50 50
Optimal x: 0.0006588658409876023, f(x): [33.8, 33.33, 32.86]
100 50
Optimal x: 0.0008366052328838717, f(x): [33.9, 33.36, 32.73]
10 100
Optimal x: 0.0007161116781374656, f(x): [33.72, 33.34, 32.93]
50 100
Optimal x: 0.0008499011081537673, f(x): [33.84, 33.34, 32.81]
100 100
Optimal x: 0.0009963889622691827, f(x): [33.85, 33.33, 32.81]


In [172]:
fww = bww = [10, 20, 30, 50, 100]
DISTR = np.ones((len(bww), len(fww)))

for iib, b in enumerate(bww):
    for iif, f in enumerate(fww):
        DISTR[iib, iif] = similarity_score(dataset_distr(b, f, "ZM", Period.MARCH, scaler=.1))

0.00041694071423861967
0.0004169388720973844
0.0004169432132866771
0.0004169527288107847
0.00041696826876724385


KeyboardInterrupt: 

In [134]:
DISTR

array([[16.84666667, 16.85333333, 16.85333333, 16.84666667, 16.84      ],
       [ 2.55333333,  2.55333333,  2.55333333,  2.55333333,  2.55333333],
       [10.79333333, 10.79333333, 10.79333333, 10.79333333, 10.79333333],
       [18.06      , 18.06      , 18.06      , 18.06      , 18.06      ],
       [23.95333333, 23.95333333, 23.95333333, 23.95333333, 23.95333333]])

In [135]:
# (W+, W-), stock, period
# a (W+, W-)

In [136]:
def count_time(v):
    diff = np.array(list(np.diff(v)) + [1])
    diff_stop = (diff != 0) * 1

    ri = 0
    count_cons = 0
    out = np.ones(diff_stop.shape)

    while ri < len(diff_stop):
        if diff_stop[ri] == 0:
            count_cons += 1
        else:
            count_cons += 1
            # print(count_cons)
            for i in range(count_cons):
                # print("out[ri-(count_cons-1)+i]", ri-(count_cons-1)+i, count_cons)
                out[ri-(count_cons-1)+i] = count_cons
            count_cons = 0
        ri += 1
    return out

In [137]:
def label_time_duration(window_size_backward, window_size_forward, stock, period, is_plot=False, gamma=120):
    exp = pd.DataFrame()
    exp["mid"] = MID_PRICES[period].loc[:,[stock]]
    exp["mid+t"] = exp["mid"].rolling(window_size_forward ).mean().shift(-window_size_forward+1)
    exp["mid-t"] = exp["mid"].rolling(window_size_backward).mean()

    exp["mid+t-inst"] = exp["mid"].shift(-window_size_forward+1)
    exp["mid-t-inst"] = exp["mid"]

    exp = exp[(window_size_backward - 1):-(window_size_forward - 1)]

    exp["change"] = (exp["mid+t"] - exp["mid-t"]) / exp["mid-t"]
    exp["ret"] = (exp["mid+t-inst"] - exp["mid-t-inst"]) / exp["mid-t-inst"]

    ratio_mu, ratio_si = exp["change"].mean(), exp["change"].std()
    avg_dist_0 = np.abs(exp["change"]).mean()

    th_u = 0 # 0+ratio_si*scaler
    th_d = 0 # 0-ratio_si*scaler

    exp["times"] = np.ones (exp.shape[0])
    exp["times"] = np.where(exp["change"] > 0, 1, exp["times"])
    exp["times"] = np.where(exp["change"] < 0, 2, exp["times"])
    exp["times"] = count_time(exp["times"])

    exp["class"] = np.ones (exp.shape[0])
    exp["class"] = np.where((exp["times"] < gamma), 1, exp["class"])
    exp["class"] = np.where((exp["times"] >= gamma) & (exp["change"] > th_u), 2, exp["class"])
    exp["class"] = np.where((exp["times"] >= gamma) & (exp["change"] < th_d), 0, exp["class"])

    COLORS = {1.0: 'gray', 2.0: 'green', 0.0: 'red'}
    LWIDTH = {1.0: 1, 2.0: 4, 0.0: 4}

    print(stock, counter_labels(exp["class"]))

    # >>>> PLOTTING <<<<<
    if is_plot:

        fig, ax = plt.subplots()
        for i in range(exp["class"].shape[0]-1)[15000:19000]:
            col = COLORS[exp["class"][i]]
            lss = LWIDTH[exp["class"][i]]

            ax.plot([i, i+1], [exp["change"][i], exp["change"][i+1]], color=col, linewidth=lss)

        ax.axhline(y=0, color="black")

        ax.set_xlabel('Time')
        ax.set_ylabel('Return')
        plt.show()

    # labels 0 (down), 1 (stable), 2 (down)
    #return counter_labels(exp["pred"])
    #return Counter(exp["pred_t1"])
    return counter_labels(exp["class"])

In [138]:
fww = bww = [10, 20, 30, 50, 100]
DISTR = np.ones((len(bww), len(STOCKS)))

for iib, b in enumerate(bww):
    print("WINDOW", b)
    for iis, s in enumerate(STOCKS):
        DISTR[iib, iis] = similarity_score(label_time_duration(b, b, s, Period.JULY, gamma=10))

# GAMMA * win * y > GAMMA'    GAMMA' prop ritardi (known 5)

WINDOW 10
AAPL [45.54, 10.93, 43.54]
TSLA [45.49, 9.03, 45.48]
ZM [45.69, 9.32, 45.0]
AAWW [29.87, 41.71, 28.41]
AGNC [11.65, 77.16, 11.19]
LYFT [43.69, 13.0, 43.31]
WINDOW 20
AAPL [49.64, 2.13, 48.23]
TSLA [48.87, 1.78, 49.34]
ZM [49.47, 1.65, 48.87]
AAWW [40.77, 18.92, 40.31]
AGNC [19.5, 61.39, 19.11]
LYFT [48.76, 2.51, 48.73]
WINDOW 30
AAPL [50.21, 0.91, 48.89]
TSLA [49.32, 0.76, 49.91]
ZM [49.98, 0.66, 49.36]
AAWW [44.93, 10.16, 44.91]
AGNC [24.95, 50.56, 24.48]
LYFT [49.69, 1.02, 49.29]
WINDOW 50
AAPL [50.65, 0.34, 49.01]
TSLA [49.72, 0.32, 49.96]
ZM [49.98, 0.25, 49.77]
AAWW [48.01, 3.97, 48.02]
AGNC [32.24, 36.0, 31.76]
LYFT [49.91, 0.36, 49.73]
WINDOW 100
AAPL [51.25, 0.08, 48.67]
TSLA [49.72, 0.06, 50.22]
ZM [49.62, 0.07, 50.31]
AAWW [49.05, 0.73, 50.22]
AGNC [41.22, 17.78, 41.0]
LYFT [49.73, 0.09, 50.18]


In [147]:
%matplotlib notebook
fig, ax = plt.subplots()
ax.imshow(DISTR, cmap='hot', interpolation='nearest')
ax.set_yticks(np.arange(len(bww)), labels=bww)
ax.set_xticks(np.arange(len(STOCKS)), labels=STOCKS)
plt.show()

<IPython.core.display.Javascript object>

In [140]:
DISTR

array([[23.07333333, 24.30666667, 24.24666667,  8.86666667, 43.98      ,
        20.46      ],
       [31.67333333, 31.70666667, 31.88      , 14.56666667, 28.18666667,
        30.83333333],
       [32.86666667, 32.76666667, 32.88      , 23.18      , 17.38666667,
        32.44666667],
       [33.54      , 33.09333333, 33.15333333, 29.36666667,  2.82666667,
        33.03333333],
       [34.11333333, 33.44      , 33.49333333, 32.99333333, 15.62666667,
        33.39333333]])

In [149]:
label_time_duration(100, 100, "AAPL", Period.JULY, gamma=10000, is_plot=True)

AAPL [0.0, 100.0, 0.0]


<IPython.core.display.Javascript object>

[0.0, 100.0, 0.0]

In [142]:
a = np.linspace(0, 50, 500)
si = pd.DataFrame(a, columns=["sin"])
si['m5'] = si["sin"].rolling(5).mean()
si['m10'] = si["sin"].rolling(40).mean()
si.plot()

<IPython.core.display.Javascript object>

<AxesSubplot: >